In [29]:
import pandas as pd

# Cargar el archivo CSV
file_path = 'C:\\Users\\Alexander\\Documents\\Data Engineering\\Data.Engineering\\Microdesafios\\Semana8\\Datos_Microdesafio_Semana8_DE.csv'
df = pd.read_csv(file_path, sep=';')

# Mostrar las primeras filas del DataFrame
print(df.head())

       Pais         Comisionado Reduccion_CO2 Incrmento_P Inversion_arboles  \
0  Argentina      Carlos Veroes            Si          Si                Si   
1   Colombia      Sofia Andrade            No          No                Si   
2      Chile  Cristina Valdivia            Si          Si                Si   
3    Bolivia       Pedro Carlos            No          No                No   
4   Paraguay      Juan Paraguas            No          Si                Si   

        Fecha  Telefono  
0  08/07/2022  23467698  
1  12/06/2022  76587899  
2  15/07/2022  76593749  
3  01/09/2022  65746474  
4  04/06/2022  83447474  


In [30]:
# Crear una copia del DataFrame para trabajar con ella
df_copy = df.copy()

# Convertir 'Fecha' a datetime
df_copy['Fecha'] = pd.to_datetime(df_copy['Fecha'], format='%d/%m/%Y')

# Convertir 'Telefono' a string (object)
df_copy['Telefono'] = df_copy['Telefono'].astype(str)

# Verificar los tipos de datos
print(df_copy.dtypes)

Pais                         object
Comisionado                  object
Reduccion_CO2                object
Incrmento_P                  object
Inversion_arboles            object
Fecha                datetime64[ns]
Telefono                     object
dtype: object


In [31]:
# Convertir la columna 'Telefono' a números enteros
df_anonymized_copy['Telefono'] = pd.to_numeric(df_anonymized_copy['Telefono'], errors='coerce')

# Inicializar el anonimizador
an = anonymize(df_anonymized_copy)

# Generar datos ficticios
fake_df = (
    an
    .fake_names("Comisionado", chaining=True)
    .fake_dates("Fecha", chaining=True)
    .fake_whole_numbers("Telefono", chaining=True)
    .show_data_frame()
)

# Verificar los datos anonimizados
print(fake_df.head())


       Pais         Comisionado Reduccion_CO2 Incrmento_P Inversion_arboles  \
0  Argentina      Carlos Veroes            Si          Si                Si   
1   Colombia      Sofia Andrade            No          No                Si   
2      Chile  Cristina Valdivia            Si          Si                Si   
3    Bolivia       Pedro Carlos            No          No                No   
4   Paraguay      Juan Paraguas            No          Si                Si   

       Fecha  Telefono      Fake_Comisionado  Fake_Fecha  Fake_Telefono  
0 2022-07-08  23467698     James Smith-Barry  2012-01-09       33706741  
1 2022-06-12  76587899          Tracy Abbott  2018-07-31       52319411  
2 2022-07-15  76593749  Ms Charlene Ferguson  2003-10-31       78020105  
3 2022-09-01  65746474        Daniel Harding  2020-11-29       47126031  
4 2022-06-04  83447474    Molly Jones-Wilson  2007-12-10       57929585  


In [34]:
fake_df.to_csv('temp_data.csv', index=False)


In [36]:
import psycopg2
import os
from dotenv import load_dotenv
import pandas as pd

# Cargar las variables de entorno desde el archivo .env
load_dotenv()

# Obtener las credenciales desde las variables de entorno
dbname = os.getenv("DB_NAME")
user = os.getenv("DB_USER")
password = os.getenv("DB_PASSWORD")
host = os.getenv("DB_HOST")
port = os.getenv("DB_PORT")

# Conectar a Redshift
conn = psycopg2.connect(
    dbname=dbname,
    user=user,
    password=password,
    host=host,
    port=port
)

# Crear un cursor
cur = conn.cursor()

# Leer el archivo CSV en un DataFrame
df = pd.read_csv('temp_data.csv')

# Insertar datos fila por fila
for _, row in df.iterrows():
    cur.execute(
        """
        INSERT INTO políticas_2050 (Pais, Comisionado, Reduccion_CO2, Incrmento_P, Inversion_arboles, Fecha, Telefono, Fake_Comisionado, Fake_Fecha, Fake_Telefono)
        VALUES (%s, %s, %s, %s, %s, %s, %s, %s, %s, %s)
        """,
        tuple(row)
    )

# Confirmar la transacción y cerrar la conexión
conn.commit()
cur.close()
conn.close()
